# Training YOLOv9e model to detect leaves

In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/WongKinYiu/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q

In [ ]:
!wget -P /kaggle/working/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt

In [ ]:
!pip install roboflow

%cd /kaggle/working/yolov9

from roboflow import Roboflow
rf = Roboflow(api_key="")  # Insert your own API Key
project = rf.workspace("workspace-mtzoa").project("leaf_noleaf")
version = project.version(2)
dataset = version.download("yolov9")

In [ ]:
%cd /kaggle/working/yolov9

!python train_dual.py \
--batch 8 --epochs 30 --img 640 --device 0 --min-items 0 --close-mosaic 15 --single-cls \
--data {dataset.location}/data.yaml \
--weights /kaggle/working/weights/yolov9-e.pt \
--cfg models/detect/yolov9-e.yaml \
--hyp hyp.scratch-high.yaml 
# \
# --resume /kaggle/working/yolov9/runs/train/exp3/weights/last.pt

In [ ]:
!python /kaggle/working/yolov9/detect_dual.py \
--img 256 --save-txt\
--weights /kaggle/working/yolov9/runs/train/exp3/weights/best.pt \
--source "/kaggle/input/plantvillage/Plant_leave_diseases_dataset_without_augmentation/Pepper,_bell___healthy/image (1).JPG"

In [ ]:
from IPython.display import Image

Image(filename=f"/kaggle/working/yolov9/runs/detect/exp/image (1).JPG", width=1000)

# Creating the Dataset

In [ ]:
import os

# Diccionario con códigos de clase
class_codes = {
    "Apple___Apple_scab": "APAS",
    "Apple___Black_rot": "APBR",
    "Apple___Cedar_apple_rust": "APCR",
    "Apple___healthy": "APHE",
    "Blueberry___healthy": "BLHE",
    "Cherry___Powdery_mildew": "CHPM",
    "Cherry___healthy": "CHHE",
    "Corn___Cercospora_leaf_spot Gray_leaf_spot": "CCGL",
    "Corn___Common_rust": "CCOR",
    "Corn___Northern_Leaf_Blight": "CNLB",
    "Corn___healthy": "CNHE",
    "Grape___Black_rot": "GRBR",
    "Grape___Esca_(Black_Measles)": "GRES",
    "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)": "GRLB",
    "Grape___healthy": "GRHE",
    "Orange___Haunglongbing_(Citrus_greening)": "ORHL",
    "Peach___Bacterial_spot": "PCBS",
    "Peach___healthy": "PCHE",
    "Pepper,_bell___Bacterial_spot": "PBBP",
    "Pepper,_bell___healthy": "PBHE",
    "Potato___Early_blight": "PTEB",
    "Potato___Late_blight": "PTLB",
    "Potato___healthy": "PTHE",
    "Raspberry___healthy": "RAHE",
    "Soybean___healthy": "SOHE",
    "Squash___Powdery_mildew": "SQPM",
    "Strawberry___Leaf_scorch": "STLS",
    "Strawberry___healthy": "STHE",
    "Tomato___Bacterial_spot": "TMBS",
    "Tomato___Early_blight": "TMEB",
    "Tomato___Late_blight": "TMLB",
    "Tomato___Leaf_Mold": "TMLM",
    "Tomato___Septoria_leaf_spot": "TMSL",
    "Tomato___Spider_mites Two-spotted_spider_mite": "TMSM",
    "Tomato___Target_Spot": "TMTS",
    "Tomato___Tomato_Yellow_Leaf_Curl_Virus": "TMYC",
    "Tomato___Tomato_mosaic_virus": "TMMV",
    "Tomato___healthy": "TMHE"
}

# Ruta de entrada y salida
input_folder = "/kaggle/input/plantvillage/Plant_leave_diseases_dataset_without_augmentation"
dataset3_images_folder = "/kaggle/working/Dataset3/images"

# Extensiones de archivo de imagen permitidas
image_extensions = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']

# Función para obtener el nombre base de la imagen sin extensión
def get_base_name(filename):
    for ext in image_extensions:
        if filename.endswith(ext):
            return filename.replace(ext, '')
    return filename

# Función para obtener todas las imágenes en una carpeta (sin extensión)
def get_images_in_folder(folder):
    images = set()
    for file in os.listdir(folder):
        if any(file.endswith(ext) for ext in image_extensions):
            images.add(get_base_name(file))
    return images

# Calcular el total de imágenes en el dataset original (excluyendo Background_without_leaves)
total_original_images = 0
for folder in os.listdir(input_folder):
    if folder != "Background_without_leaves":
        folder_path = os.path.join(input_folder, folder)
        if os.path.isdir(folder_path):
            total_original_images += len(get_images_in_folder(folder_path))

# Calcular el total de imágenes en el dataset de llegada
total_arrival_images = len(get_images_in_folder(dataset3_images_folder))
total_missing_images = 0

# Verificar imágenes faltantes para cada clase
for class_name, class_code in class_codes.items():
    original_class_folder = os.path.join(input_folder, class_name)
    if not os.path.exists(original_class_folder):
        print(f"Carpeta original {original_class_folder} no existe, saltando...")
        continue
    
    original_images = get_images_in_folder(original_class_folder)
    original_image_count = len(original_images)
    
    dataset3_images = get_images_in_folder(dataset3_images_folder)
    dataset3_images = {img.replace(f"{class_code}_", "") for img in dataset3_images if img.startswith(class_code)}
    arrival_image_count = len(dataset3_images)
    
    missing_images = original_images - dataset3_images
    missing_count = len(missing_images)
    total_missing_images += missing_count
    
    print(f"Clase {class_name}:")
    print(f"  Total de imágenes en carpeta original: {original_image_count}")
    print(f"  Total de imágenes en el dataset de llegada: {arrival_image_count}")
    print(f"  Imágenes faltantes ({missing_count} imágenes):")
    if missing_images:
        for img in sorted(missing_images):
            print(f"    {img}")
    else:
        print(f"    No faltan imágenes.")

print(f"\nResumen General:")
print(f"  Total de imágenes en el dataset original (excluyendo Background): {total_original_images}")
print(f"  Total de imágenes en el dataset de llegada: {total_arrival_images}")
print(f"  Total de imágenes faltantes: {total_missing_images}")
